# Preparation for distilation
## calculate the baseline

## loade model & data

In [1]:
RUNPPL = True
CLEANTEACHER = False
LOADRUNPPL = False
ONLYTRAINMPO = True
SAVEMODEL = True
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
from huggingface_hub import login

login(token="")  # 替换为你的 Hugging Face 访问令牌

/home/zhaohongzhicai/miniconda3/envs/qwen310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

import torch, transformers, huggingface_hub, tokenizers
print("Python:",   torch.__spec__.origin.split("python")[1].split("/")[0])
print("Torch:",    torch.__version__)
print("Transformers:", transformers.__version__)
print("Hub:", huggingface_hub.__version__)
print("Tokenizers:", tokenizers.__version__)


Python: 3.10
Torch: 2.9.1+cu128
Transformers: 4.57.1
Hub: 0.36.0
Tokenizers: 0.22.1


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from datasets import load_dataset

dataset = load_dataset("Nan-Do/code-search-net-python", split="train").select(range(20000)) # 选择部分样本用于蒸馏 20000
dataset2 = load_dataset("Nan-Do/code-search-net-python", split="train").select(range(20001,21001)) # 1000 样本用于测试 PPL

device = torch.device("cuda:1")

# student_device = torch.device("cuda:0")
# teacher_device = torch.device("cuda:0")



/home/zhaohongzhicai/miniconda3/envs/qwen310/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  # 引入 BitsAndBytesConfig
import torch
QWEN2point5_1point5B_MODEL_NAME = "Qwen/Qwen2.5-1.5B"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(QWEN2point5_1point5B_MODEL_NAME, trust_remote_code=True)

# model_1_5b = AutoModelForCausalLM.from_pretrained(
#     QWEN2point5_1point5B_MODEL_NAME,
#     torch_dtype=torch.bfloat16,          # 新版 transformers 用 dtype，就按 warning 说的
#     trust_remote_code=True,
#     device_map=device
# )
model_1_5b = AutoModelForCausalLM.from_pretrained(
    QWEN2point5_1point5B_MODEL_NAME,
    quantization_config=bnb_config,  # <--- 传入量化配置
    torch_dtype=torch.bfloat16,      
    trust_remote_code=True,
    device_map=device                # 或者是 "auto"
)           
teacher = model_1_5b
print(teacher.device)              # 确认一下
tokenizer = AutoTokenizer.from_pretrained(QWEN2point5_1point5B_MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


`torch_dtype` is deprecated! Use `dtype` instead!


cuda:1


In [6]:
from tqdm import tqdm
import torch

def compute_ppl_batch(model, tokenizer, dataset, num_samples=100, target_gpu=torch.device("cuda:0")):
    total_ppl = 0.0
    device = target_gpu
    for i in tqdm(range(num_samples)):
        text = dataset[i]["code"]
        enc = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            out = model(**enc, labels=enc["input_ids"])
        loss = out.loss
        total_ppl += torch.exp(loss).item()
    return total_ppl / num_samples
if RUNPPL:
    print(compute_ppl_batch(model_1_5b, tokenizer, dataset2, target_gpu=device))

  0%|          | 0/100 [00:00<?, ?it/s]/home/zhaohongzhicai/miniconda3/envs/qwen310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 100/100 [00:13<00:00,  7.61it/s]

9.09707392692566


3.9,6.2

In [7]:
if CLEANTEACHER:
    del model_1_5b
    del tokenizer
    import gc
    gc.collect() 
    torch.cuda.empty_cache()

In [8]:


QWEN2point5_0point5B_MODEL_NAME = "Qwen/Qwen2.5-0.5B"


model_0_5b = AutoModelForCausalLM.from_pretrained(
    QWEN2point5_0point5B_MODEL_NAME,
    # device_map="cuda:0",# for PPL
    device_map=device, 
    dtype=torch.float32,
    trust_remote_code=True
)



In [9]:
if RUNPPL:
    print(compute_ppl_batch(model_0_5b, tokenizer, dataset2,target_gpu=device))

100%|██████████| 100/100 [00:03<00:00, 30.80it/s]

10.877563061714172


In [ ]:
# prompt = "Write a short Python function to compute Fibonacci numbers."
# inputs = tokenizer(prompt, return_tensors="pt").to(model_0_6b.device)
# with torch.no_grad():
#     outputs = student.generate(**inputs, max_new_tokens=200)
# print(tokenizer_0_6b.decode(outputs[0], skip_special_tokens=True))

In [11]:
from transformers import AutoConfig

def print_model_layers(model_name):
    try:
        config = AutoConfig.from_pretrained(
            model_name,
            trust_remote_code=True
        )
    except Exception as e:
        print(f"Error loading config for {model_name}: {e}")
        return

    if hasattr(config, "num_hidden_layers"):
        num_layers = config.num_hidden_layers
        print("--- Model Layers ---")
        print(f"模型名称: {model_name}")
        print(f"Transformer 层数: {num_layers}")
    else:
        print(f"在 {model_name} 的配置中未找到 'num_hidden_layers' 属性。")

print_model_layers(QWEN2point5_1point5B_MODEL_NAME)
print(model_1_5b)
print("-" * 20)
print_model_layers(QWEN2point5_0point5B_MODEL_NAME)
print(model_0_5b)


--- Model Layers ---
模型名称: Qwen/Qwen2.5-1.5B
Transformer 层数: 28
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernor

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc  # 引入垃圾回收模块

class OverParameterizedLinear(nn.Module):
    def __init__(self, original_linear, expansion_factor=1.5):
        super().__init__()
        self.in_features = original_linear.in_features
        self.out_features = original_linear.out_features
        self.bias = original_linear.bias
        
        # 计算中间维度
        self.mid_dim = int(min(self.in_features, self.out_features) * expansion_factor)
        
        device = original_linear.weight.device
        dtype = original_linear.weight.dtype

        # ============================================================
        # 优化版 SVD 初始化 (Asymmetric SVD + Error Check)
        # ============================================================
        
        # 1. 强制使用 float32 进行高精度分解
        print(f"  > Initializing MPO layer: {self.in_features} -> {self.mid_dim} -> {self.out_features}")
        w_orig_cpu = original_linear.weight.data.float().cpu()
        
        # 2. SVD 分解
        U, S, Vh = torch.linalg.svd(w_orig_cpu, full_matrices=False)
        
        # 3. 截断与分配 (非对称分配，减少 sqrt 带来的精度损失)
        r = min(self.in_features, self.out_features)
        
        # 方案：A = U, B = S @ Vh
        # 这样 A 是正交矩阵，数值非常稳定；B 承载了幅值
        A_init = U[:, :r] 
        B_init = torch.diag(S[:r]) @ Vh[:r, :]
        
        # 4. 初始化参数
        # A: 随机初始化 (提供梯度路径)
        # B: 全 0 初始化 (关键！确保 expansion 部分对结果无影响)
        self.matrix_a = nn.Parameter(torch.randn(self.out_features, self.mid_dim, device=device) * 0.001)
        self.matrix_b = nn.Parameter(torch.zeros(self.mid_dim, self.in_features, device=device))
        
        # 5. 填入 SVD 结果
        with torch.no_grad():
            # 将 numpy/cpu 结果转回目标设备和精度
            self.matrix_a.data[:, :r] = A_init.to(device=device, dtype=dtype)
            self.matrix_b.data[:r, :] = B_init.to(device=device, dtype=dtype)
            
            # === 自检环节：验证初始化误差 ===
            # 计算重构矩阵
            W_recon = self.matrix_a.data @ self.matrix_b.data
            W_orig = original_linear.weight.data
            
            # 计算相对误差 (Frobenius Norm)
            diff = (W_recon - W_orig).float().norm()
            total = W_orig.float().norm()
            rel_error = diff / total
            
            print(f"  > Reconstruction Error: {rel_error.item():.6f}")
            if rel_error > 1e-2: # 如果误差大于 1%，说明初始化失败
                print("  !!! WARNING: High reconstruction error. Check dtype or SVD logic.")

        # 6. 清理
        del w_orig_cpu, U, S, Vh, A_init, B_init, W_recon
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        # 同步精度
        self.to(device, dtype=dtype)

    def forward(self, x):
        x_mid = F.linear(x, self.matrix_b)
        out = F.linear(x_mid, self.matrix_a, self.bias)
        return out

    @torch.no_grad()
    def merge_to_linear(self):
        merged_weight = self.matrix_a @ self.matrix_b
        new_linear = nn.Linear(self.in_features, self.out_features, bias=self.bias is not None)
        new_linear.weight.data = merged_weight
        if self.bias is not None:
            new_linear.bias.data = self.bias
        return new_linear
    
# 2. 专门针对 Qwen2.5 的替换函数
def apply_mpo_to_qwen(model, num_layers_to_replace=2, expansion_factor=1.5):
    """
    只替换 Qwen 模型最后几层的 MLP。
    """
    # Qwen 的层列表通常在 model.model.layers
    layers = model.model.layers
    total_layers = len(layers)
    start_layer = total_layers - num_layers_to_replace
    
    print(f">>> 正在将 Layer {start_layer} 到 Layer {total_layers-1} 的 MLP 转换为超参数化层...")
    print(f">>> 扩展倍率: {expansion_factor}x")

    for i in range(start_layer, total_layers):
        layer = layers[i]
        # Qwen2.5 的 MLP 包含三个线性层: gate_proj, up_proj, down_proj
        # 我们全部替换，或者为了省显存只替换 down_proj (它是输出层，最关键)
        
        # 这里演示全部替换 (最强效果)
        # 注意：gate_proj 和 up_proj 维度通常很大，如果显存不够，可以把 expansion_factor 调小
        layer.mlp.gate_proj = OverParameterizedLinear(layer.mlp.gate_proj, expansion_factor)
        layer.mlp.up_proj = OverParameterizedLinear(layer.mlp.up_proj, expansion_factor)
        layer.mlp.down_proj = OverParameterizedLinear(layer.mlp.down_proj, expansion_factor)
        
    print(">>> 替换完成。")
    return model
def merge_mpo_back_to_qwen(model):
    """
    遍历模型，将所有的 OverParameterizedLinear 层合并回标准的 nn.Linear。
    这是为了：
    1. 恢复模型原始结构，使其能被 AutoModel 加载。
    2. 移除超参数化带来的额外参数，恢复推理速度。
    """
    print(">>> 开始将 MPO 层合并回标准 Linear 层...")
    
    # 遍历 Qwen 的所有层
    for i, layer in enumerate(model.model.layers):
        # 检查 MLP 的三个投影层
        for proj_name in ["gate_proj", "up_proj", "down_proj"]:
            # 获取当前模块 (可能是 nn.Linear 也可能是 OverParameterizedLinear)
            current_module = getattr(layer.mlp, proj_name)
            
            # 如果是我们要找的超参数化层
            if isinstance(current_module, OverParameterizedLinear):
                print(f"  - Merging Layer {i} MLP {proj_name}...")
                
                # 调用类内部的合并方法，得到标准的 nn.Linear
                merged_linear = current_module.merge_to_linear()
                
                # 将原来的模块替换为合并后的模块
                setattr(layer.mlp, proj_name, merged_linear)
                
    print(">>> 合并完成。模型已恢复原始架构。")
    return model

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, get_scheduler
from dataclasses import dataclass
from tqdm.auto import tqdm
import torch.nn.functional as F



@dataclass
class DistillConfig:
    temperature: float = 2.0
    alpha_kd: float = 0.5       # logits KD 权重 λ_kd 0.5
    alpha_feat: float = 0.1     # hidden feature 对齐权重 λ_feat
    alpha_param: float = 0.0    # 参数重构正则权重 λ_param（先关掉）
    use_feat_loss: bool = True  # 是否使用特征蒸馏（只对齐最后一层）
    use_param_loss: bool = False  # 是否使用参数正则（权重对齐）

distill_cfg = DistillConfig(
    temperature=2.0,
    alpha_kd=0.5,
    alpha_feat=1,
    alpha_param=0.0,          # 默认不打开参数正则，有需要再开
    use_feat_loss=True,
    use_param_loss=False,
)



teacher = model_1_5b               # 假设已经是 int8 + device_map={0,1}
student = apply_mpo_to_qwen(model_0_5b, num_layers_to_replace=2, expansion_factor=1.5)# MPO 学生模型（Qwen 0.6B MPO 版）
# teacher = teacher.to(teacher_device)
# student = student.to(student_device)
student_dtype = student.dtype
# ... apply_mpo_to_qwen ...

# 冒烟测试：确保初始状态没崩
print(">>> Running Sanity Check (Initial PPL)...")
dataset_check = load_dataset("Nan-Do/code-search-net-python", split="train").select(range(20001,21001)) # 只测50条
student.eval()
ppl_check = compute_ppl_batch(student, tokenizer, dataset_check, target_gpu=device)
print(f">>> Initial PPL: {ppl_check}") 
# 预期：这里必须接近 6.83。如果是 8.x 或 16.x，说明初始化还是有问题，不要开始训练。

# ... 确认无误后，开始 optimizer 定义和 training loop ...

# teacher 不更新
teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)


max_len = 256
batch_size = 1
gradient_accumulation_steps = 32
num_epochs = 1
lr = 1e-5

raw_train = dataset
def tokenize_fn(batch):
    out = tokenizer(
        batch["code"],
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_attention_mask=True,
    )
    # 非 padding token 才参与 CE
    out["labels"] = [
        [tid if m == 1 else -100 for tid, m in zip(ids, mask)]
        for ids, mask in zip(out["input_ids"], out["attention_mask"])
    ]
    return out

train_ds = raw_train.map(
    tokenize_fn,
    batched=True,
    remove_columns=raw_train.column_names,
)
train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collator,
    num_workers=0,
    pin_memory=True,
)

# =====================================================
# projection（teacher hidden -> student hidden）
# =====================================================

teacher_hidden = teacher.config.hidden_size
student_hidden = student.config.hidden_size

if teacher_hidden != student_hidden:
    projector = nn.Linear(
        teacher_hidden,
        student_hidden,
        bias=False,
        device=device,
        dtype=student_dtype,
    )
else:
    projector = nn.Identity().to(device)

# =====================================================
# 参数正则：只针对一小部分层（示例：lm_head）
# 先构造一个 param_pairs 列表，用于 L_param
# =====================================================

param_pairs = []
if distill_cfg.use_param_loss and distill_cfg.alpha_param > 0.0:
    # 示例：对齐 lm_head 权重（按你模型的真实名字来改）
    with torch.no_grad():
        t_lm_head_w = teacher.lm_head.weight.to(device, dtype=student_dtype)
    param_pairs.append((student.lm_head.weight, t_lm_head_w))

# =====================================================
# 定义各个 loss 模块
# =====================================================

mse = nn.MSELoss()

def compute_ce_loss(student_output):
    """
    L_CE：student 对真实 token 的交叉熵。
    HF 的 causal LM 模型在 forward 时已经返回了 loss。
    """
    return student_output.loss

def compute_kd_loss(student_logits, teacher_logits, T, attention_mask):
    """
    修正版：只计算非 Padding 部分的 KD Loss
    """
    # 1. 计算软标签概率
    s = (student_logits / T).log_softmax(dim=-1)
    t = (teacher_logits / T).softmax(dim=-1)
    
    # 2. 计算逐个 Token 的 KL 散度 [Batch, Seq_Len]
    # KL = sum(p * (log_p - log_q)) = sum(p * log_p - p * log_q)
    # 这里用简化的 sum(-t * s) 形式（忽略 t*log_t 常数项对梯度的影响）
    kl_pointwise = torch.sum(-t * s, dim=-1) 
    
    # 3. 应用 Mask [Batch, Seq_Len]
    # attention_mask 形状通常是 [Batch, Seq_Len]，1为有效，0为Pad
    mask = attention_mask.to(kl_pointwise.dtype)
    
    # 4. 只取有效 Token 的平均值
    loss = (kl_pointwise * mask).sum() / mask.sum().clamp_min(1.0)
    
    return loss * (T * T)


def compute_feat_loss(s_last_hidden, t_last_hidden_projected, attention_mask=None):
    # 对特征进行 L2 归一化，消除模长影响，只关注方向一致性
    s_norm = F.normalize(s_last_hidden, p=2, dim=-1)
    t_norm = F.normalize(t_last_hidden_projected, p=2, dim=-1)
    
    if attention_mask is None:
        return mse(s_norm, t_norm)

    mask = attention_mask.unsqueeze(-1).to(s_last_hidden.dtype)
    s_masked = s_norm * mask
    t_masked = t_norm * mask
    
    # 归一化后的 MSE，数值通常很小 (<1.0)
    denom = mask.sum().clamp_min(1.0)
    # 注意：归一化后 MSE = 2 * (1 - cosine_similarity)
    return ((s_masked - t_masked) ** 2).sum() / denom

def compute_param_loss(param_pairs):
    """
    L_param：参数正则（权重对齐）。
    param_pairs: List[(W_student, W_teacher_fixed)]
    通常只选很少的一两层做（例如 lm_head 或某几层 MPO）。
    """
    if not param_pairs:
        return torch.tensor(0.0, device=device)

    loss = torch.tensor(0.0, device=device)
    for w_s, w_t in param_pairs:
        loss = loss + ((w_s - w_t) ** 2).mean()
    return loss

# =====================================================
# 优化器 & Scheduler
# =====================================================
num_update_steps_per_epoch = len(train_loader) // gradient_accumulation_steps
num_training_steps = num_epochs * num_update_steps_per_epoch

if ONLYTRAINMPO:
    # 1. 先冻结 Student 的所有参
    for param in student.parameters():
        param.requires_grad = False

    # 2. 遍历模型，解冻 OverParameterizedLinear 层
    trainable_params_count = 0
    all_params_count = 0

    for name, module in student.named_modules():
        # 如果是我们在 MPO 替换中加入的层
        if isinstance(module, OverParameterizedLinear):
            # 解冻 A 和 B
            module.matrix_a.requires_grad = True
            module.matrix_b.requires_grad = True
            # 如果有 bias 也解冻
            if module.bias is not None:
                module.bias.requires_grad = True
                
            print(f"Unlocked MPO layer: {name}")

    # 3. Projector 必须训练
    for param in projector.parameters():
        param.requires_grad = True

    # 统计一下可训练参数量
    for param in student.parameters():
        all_params_count += param.numel()
        if param.requires_grad:
            trainable_params_count += param.numel()
            
    print(f"Trainable Params: {trainable_params_count / 1e6:.2f}M / {all_params_count / 1e6:.2f}M")
    print(f"Ratio: {trainable_params_count / all_params_count * 100:.2f}%")

    # 4. 重新定义 Optimizer (只包含 requires_grad=True 的参数)
    optimizer = torch.optim.AdamW(
        list(filter(lambda p: p.requires_grad, student.parameters())) + list(projector.parameters()),
        lr=5e-5
    )

    # Scheduler 保持不变
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=num_training_steps,
    )
else:
    optimizer = torch.optim.AdamW(
        list(student.parameters()) + list(projector.parameters()),
        lr=lr, # 建议保持 1e-5 或稍微降低到 5e-6
    )

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=100,            # <--- 在这里设置 Warmup
        num_training_steps=num_training_steps,
    )

# =====================================================
# 训练循环：双 GPU 蒸馏
#   - teacher 用 device_map 多卡，只出 logits (+ 可选 hidden)
#   - student 固定在 cuda:1
# =====================================================

student.train()
projector.train()

progress = tqdm(range(num_training_steps))
# 确保在循环开始前清空梯度
optimizer.zero_grad()

for epoch in range(num_epochs):
    for step, batch in enumerate(train_loader):
        # ... (数据搬运部分保持不变) ...
        batch_cpu = {k: v for k, v in batch.items()}
        batch_student = {k: v.to(device, non_blocking=True) for k, v in batch_cpu.items()}
        attention_mask = batch_student["attention_mask"]

        # ... (Teacher Forward 保持不变) ...
        with torch.no_grad():
            batch_teacher = {
                k: v.to(device, non_blocking=True)
                for k, v in batch_cpu.items()
                if k in ("input_ids", "attention_mask")
            }
            t_out = teacher(
                **batch_teacher,
                output_hidden_states=distill_cfg.use_feat_loss,
                use_cache=False,
            )
            t_logits = t_out.logits
            t_last_hidden = t_out.hidden_states[-1] if distill_cfg.use_feat_loss else None

        # ... (Student Forward 保持不变) ...
        s_out = student(
            **batch_student,
            output_hidden_states=distill_cfg.use_feat_loss,
            use_cache=False,
        )
        s_logits = s_out.logits
        s_last_hidden = s_out.hidden_states[-1] if distill_cfg.use_feat_loss else None

        # ... (Loss 计算保持不变) ...
        loss_ce = compute_ce_loss(s_out)
        
        # 修正版 KD (带 mask)
        loss_kd = compute_kd_loss(
            s_logits, 
            t_logits, 
            T=distill_cfg.temperature, 
            attention_mask=attention_mask
        )

        # 归一化版 Feat
        if distill_cfg.use_feat_loss and t_last_hidden is not None:
            # === 核心修改：强制转换 Teacher 的输出类型以匹配 Projector ===
            t_last_hidden_cast = t_last_hidden.to(dtype=projector.weight.dtype) 
            
            t_last_proj = projector(t_last_hidden_cast)
            
            loss_feat = compute_feat_loss(
                s_last_hidden,
                t_last_proj,
                attention_mask=attention_mask,
            )
        else:
            loss_feat = torch.tensor(0.0, device=device)

        if distill_cfg.use_param_loss:
            loss_param = compute_param_loss(param_pairs)
        else:
            loss_param = torch.tensor(0.0, device=device)

        # 总 Loss
        loss = (
            loss_ce
            + distill_cfg.alpha_kd * loss_kd
            + distill_cfg.alpha_feat * loss_feat
            + distill_cfg.alpha_param * loss_param
        )

        if torch.isnan(loss):
            print(f"Skipping step {step} due to NaN loss")
            continue

        # ---------- 6. 反向 & 更新 (逻辑修正) ----------
        
        # 1. Loss 归一化 (分摊到每一步)
        loss = loss / gradient_accumulation_steps
        
        # 2. 反向传播 (每一步都做，梯度会累积在 .grad 中)
        loss.backward()

        # 3. 参数更新 (只在累积够了步数后做)
        if (step + 1) % gradient_accumulation_steps == 0:
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
            
            # 更新参数
            optimizer.step()
            
            # 更新学习率
            lr_scheduler.step()
            
            # 清空梯度 (为下一轮累积做准备)
            optimizer.zero_grad()
            
            # 更新进度条
            progress.update(1)
            progress.set_postfix({
                "loss": loss.item() * gradient_accumulation_steps, # 还原数值方便观察
                "ce": loss_ce.item(),
                "kd": loss_kd.item(),
                "feat": loss_feat.item(),
                "lr": optimizer.param_groups[0]['lr']
            })


del teacher
import gc
gc.collect() 
torch.cuda.empty_cache()

student.eval()

ppl = compute_ppl_batch(student, tokenizer, dataset2, target_gpu=device)
print(f"Final PPL before merging: {ppl}")
student = merge_mpo_back_to_qwen(student)
ppl = compute_ppl_batch(student, tokenizer, dataset2, target_gpu=device)
print(f"Final PPL after merging: {ppl}")


>>> 正在将 Layer 22 到 Layer 23 的 MLP 转换为超参数化层...
>>> 扩展倍率: 1.5x
  > Initializing MPO layer: 896 -> 1344 -> 4864
  > Reconstruction Error: 0.000002
  > Initializing MPO layer: 896 -> 1344 -> 4864
  > Reconstruction Error: 0.000002
  > Initializing MPO layer: 4864 -> 1344 -> 896
  > Reconstruction Error: 0.000002
  > Initializing MPO layer: 896 -> 1344 -> 4864
  > Reconstruction Error: 0.000002
  > Initializing MPO layer: 896 -> 1344 -> 4864
  > Reconstruction Error: 0.000002
  > Initializing MPO layer: 4864 -> 1344 -> 896
  > Reconstruction Error: 0.000002
>>> 替换完成。
>>> Running Sanity Check (Initial PPL)...


100%|██████████| 100/100 [00:03<00:00, 30.88it/s]


>>> Initial PPL: 10.87755845785141
Unlocked MPO layer: model.layers.22.mlp.gate_proj
Unlocked MPO layer: model.layers.22.mlp.up_proj
Unlocked MPO layer: model.layers.22.mlp.down_proj
Unlocked MPO layer: model.layers.23.mlp.gate_proj
Unlocked MPO layer: model.layers.23.mlp.up_proj
Unlocked MPO layer: model.layers.23.mlp.down_proj
Trainable Params: 46.45M / 514.33M
Ratio: 9.03%


100%|██████████| 100/100 [00:03<00:00, 29.23it/s]t, loss=19.3, ce=1.76, kd=34.7, feat=0.248, lr=0]       


Final PPL before merging: 8.042026400566101
>>> 开始将 MPO 层合并回标准 Linear 层...
  - Merging Layer 22 MLP gate_proj...
  - Merging Layer 22 MLP up_proj...
  - Merging Layer 22 MLP down_proj...
  - Merging Layer 23 MLP gate_proj...
  - Merging Layer 23 MLP up_proj...
  - Merging Layer 23 MLP down_proj...
>>> 合并完成。模型已恢复原始架构。


100%|██████████| 100/100 [00:03<00:00, 29.86it/s]

Final PPL after merging: 8.042026131153106


In [14]:
if SAVEMODEL:
    save_dir = "distilled_qwen2_0_5b_mpo_kd_modular"
    student.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)

In [15]:
print(num_update_steps_per_epoch)

625


In [16]:
if LOADRUNPPL:
        
    import os, glob
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    from datasets import load_dataset

    save_dir = "distilled_qwen2_0_5b_kd_modular"  # 替换为你的保存目录

    # 1) Inspect the save directory for safetensors/pytorch files
    files = glob.glob(os.path.join(save_dir, "*"))
    print("Saved files:", files)
    safetensors_files = [f for f in files if f.endswith(".safetensors")]
    if safetensors_files:
        print("Found safetensors files:", safetensors_files)
    else:
        print("No .safetensors files found. Looking for pytorch weights...")
        pt_files = [f for f in files if "pytorch_model" in f or f.endswith(".bin")]
        print("Pytorch files:", pt_files)

    # 2) Load the tokenizer
    tokenizer_distilled = AutoTokenizer.from_pretrained(save_dir, trust_remote_code=True)
    if tokenizer_distilled.pad_token is None:
        tokenizer_distilled.pad_token = tokenizer_distilled.eos_token

    # 3) Load the model (AutoModelForCausalLM handles safetensors automatically if present)
    # Use device_map to put weights on the same device as `device` variable defined in the notebook.
    # If you want to load to CPU first, set device_map="cpu"
    try:
        model_distilled = AutoModelForCausalLM.from_pretrained(
            save_dir,
            trust_remote_code=True,
            device_map=device,   # uses device variable from the notebook (e.g., "cuda:0")
            low_cpu_mem_usage=True
        )
    except Exception as e:
        print("Loading with device_map=device failed:", e)
        print("Retrying with device_map='auto'...")
        model_distilled = AutoModelForCausalLM.from_pretrained(
            save_dir,
            trust_remote_code=True,
            device_map="auto",
            low_cpu_mem_usage=True
        )
    dataset = load_dataset("Nan-Do/code-search-net-python", split="train").select(range(10000,11000)) # 1000 样本用于测试 PPL
    print(compute_ppl_batch(model_distilled, tokenizer_distilled, dataset))



In [17]:
# loss = alpha_kd * loss_kd + alpha_feat * feat_loss + (1 - alpha_kd - alpha_feat) * loss_lm


In [18]:
prompt = "Write a short Python function to compute Fibonacci numbers."
inputs = tokenizer(prompt, return_tensors="pt").to(student.device)
with torch.no_grad():
    outputs = student.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Write a short Python function to compute Fibonacci numbers. def fibonacci(n):
    if n < 0:
        print("Incorrect input")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
